# Annotating the data
For the annotations of the sample I use the quantative content analysis (Lamnek 2005). Here three categories will be formed:
1. non-answer: The category encompasses every response where no reaction to the question occurs. Example: ""
2. evasive answer: This category is defined as reacting to the question in not or just partly answering the question. Example: "Sehr geehrter Herr W., haben Sie vielen Dank für Ihre Anfrage. Ich beteilige mich nicht länger am Portal abgeordnetenwatch.de. Um Ihre Frage dennoch zu beantworten, bitte ich um Mitteilung Ihrer E-Mail-Adresse an antje.tillmann@bundestag.de. Mit freundlichen Grüßen Antje Tillmann MdB"
3. answer: Every response which contains the answer to the questions in annotated in this category. Expample: "Sehr geehrter Herr Schellerich,die gesamte Fraktion DIE LINKE im Deutschen Bundestag wird dem ESM-Vertrag nicht zustimmen. Ich habe dies in meiner Rede vom 29.März im Bundestag auch versucht zu begründen. Mit freundlichen Grüßen Dr. Gysi"

The drawn sample will be mannualy annotated. Next the sample will be used to categorise the rest of the answers automatically.

In [2]:
# load libraries for data manipulation
import pandas as pd
import re
import regex
import numpy as np

# ML: Train/test splits, cross validation,
# gridsearch
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
)

# load libraries for tokenization
import nltk
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
#nltk.download("stopwords")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing

# load libraries for text cleaning
import spacy
import ufal.udpipe
from gensim.models import KeyedVectors, Phrases
from gensim.models.phrases import Phraser
from ufal.udpipe import Model, Pipeline
import conllu

# Supervised text classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import joblib
#import eli5



## Preprocessing

In [4]:
# load data
sample_df = pd.read_csv("./data/stratified_sample.csv")

sample_df_2 = pd.read_csv("./data/stratified_sample_2.csv", sep=";")

sample_df = pd.concat([sample_df, sample_df_2])

In [5]:
# remove NaN for tokenizer to work
sample_df = sample_df.dropna(subset=["answer"])

In [6]:
sample_df = sample_df.drop_duplicates(["answer", "question_text", "party", "first_name", "last_name", "question_teaser"])

The next step comprises the preprocessing of the data. All answers will be converted to lowercase, punctuation and other noise will be removed. Lowercasing each word has the advantage that there no two different writing styles of a word. I.e. "die" and "Die" are now recognized as the same word.

In [7]:
# lower the answers to make the analysis easier
columns_to_process = ["answer", "question_text", "question_teaser"]
sample_df[columns_to_process] = sample_df[columns_to_process].apply(lambda col: col.str.lower())

# remove links and punctuation
sample_df[columns_to_process] = sample_df[columns_to_process].apply(lambda col: col.str.replace(r"\bhttps?://\S*|&\w+;|[\.,]", " ", regex=True))
sample_df[columns_to_process] = sample_df[columns_to_process].apply(lambda col: col.str.replace(r"\s+", " ", regex=True))

# combine the columns "answer", "question_text" and "question_teaser" to one column
sample_df["combined"] = sample_df[["question_text", "question_teaser"]].apply(lambda row: " ".join(row.values.astype(str)), axis=1)

## Analyzing

In the next code chunk the sample data will be split into a training and test set. On the data of the training set the model will train and with the testing set the trained model will be tested. This step is necessary to avoid overfitting and ensure the quality of the results. This classifier functions as a baseline.

### Creating a pipeline
In the next step different pipelines are created to efficiently test and tune different vectorizers and classifiers. First the CountVectorizer(), the TfidfVectorizer(), the MultinomialNB() and the LogisticRegression() are used.

In [ ]:
# split data into training and testing set with a testing set size of 20% of the data.
X_train, X_test, y_train, y_test = train_test_split(
    sample_df["answer"],
    sample_df["answer_encoded"],
    test_size=0.2,
    random_state=42
)

pipes_and_grids = [
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", MultinomialNB())
            ]
        ),
        "grid" : {
                    "vectorizer__stop_words" : [None, stopwords.words("german")],
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }     
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", TfidfVectorizer()),
                ("classifier", MultinomialNB())
            ]
        ),
        "grid" : {
                    "vectorizer__stop_words" : [None, stopwords.words("german")],
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", LogisticRegression(solver="lbfgs"))
            ]
        ),
        "grid" : {
                    "vectorizer__stop_words" : [None, stopwords.words("german")],
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
                    "classifier__C" : [0.01, 1, 100]
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", TfidfVectorizer()),
                ("classifier", LogisticRegression(solver="lbfgs"))
            ]
        ),
        "grid" : {
                    "vectorizer__stop_words" : [None, stopwords.words("german")],
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
                    "classifier__C" : [0.01, 1, 100]
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", RandomForestClassifier(n_estimators=100))
            ]
        ),
        "grid" : {
                    "vectorizer__stop_words" : [None, stopwords.words("german")],
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }
    }
]

for pipe_and_grid in pipes_and_grids:
    search = GridSearchCV(
        estimator=pipe_and_grid["pipeline"], n_jobs=-1, param_grid=pipe_and_grid["grid"], scoring="accuracy", cv=10
    )

    search.fit(X_train, y_train)
    
    pred = search.predict(X_test)

    print(f"Vectorizer and classifier: {pipe_and_grid['pipeline']}")
    print(f"Best parameters: {search.best_params_}")

    rep = metrics.classification_report(y_test, pred)
    print(rep)

- stemming
- svm, knearest neighbors, gradient boost
- word embeddings
- question_text/_teaser

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_df["answer"],
    sample_df["answer_encoded"],
    test_size=0.2,
    random_state=42
)

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    ngram_range=(1,3)
)

text_train = vectorizer.fit(X_train)
text_test = vectorizer.fit(X_test)

scaler = preprocessing.StandardScaler().fit(text_train)




ValueError: could not convert string to float: 'sehr geehrter herr r vielen dank für ihre frage!die auskunftspflicht nach § 36 absatz 3 infektionsschutzgesetz gilt nur für die in § 36 absatz 1 und absatz 2 des infektionsschutzgesetzes genannten einrichtungen und unternehmen aus dem grunde weil wir es dort mit besonders gefährdeten personen zu tun haben (bspw in kitas heimen pflegeeinrichtungen) daher ist der impfstatus der abgeordneten nicht statistisch erfasst unter ist ersichtlich dass im bundestag die 3-g-regelung umgesetzt wird hier ist auch festgehalten: "die im rahmen von nummern 3 bis 8 erhobenen personenbezogenen daten dürfen ausschließlich zum zweck der zutrittskontrollen verarbeitet werden sie werden nicht gespeichert "alles gute und freundliche grüße team martin dulig'